Предоставлено 2 решения: "везучий" GBM и "настроенный" XGB. В первом варианте не используется почти никаких приёмов работы с выборкой, но он показывает лучшие результаты вопреки внутренним тестам.

Исходный код: GBM, XGB и работа с тестовой выборкой.

### Метрики качества
1. 3-fold кросс-валидация без учета времени.
2. Кросс-валидация с накоплением обучающей части от времени
3. SMAPE на извлечённых из тестовой выборки ответах. Если бы не было, контрольную часть можно было сделать из ~10% обучающей.

Все довольно неплохо кореллировало с результатами соревнования, пока дело не касалось борьбы за 1-2 пункта. Там начинались необъяснимые несоотвествия в любую сторону. На внутренних тестах GBM проигрывал XGB, причем значительно.

### Работа с метрикой
Нужно было заставить алгоритмы оптимизировать SMAPE, не MSE.

1. Извлечение корня из y позволяет немного приблизить значения друг к другу и сравнять штрафы за большой и маленький размер числа. Оптимально извлекать 2-4 степень, иначе **y** становятся слишком близкими и не обучают алгоритм. Значительно улучшает качество (естественно, если не забыть возвести обратно)
   * Использовалось обоими алгоритмами
2. GBM поддерживает оптимизацию MAE "из коробки". Это по-прежнему не нормирует ответы, но хотя бы штрафы не квадратичные. Значительно улучает качество.
   * В итоге не использовалось
3. XGB можно прописать функцию, указав её градиент.
   * Прописать SMAPE не удалось, потому что производные очень маленькие и алгоритм упирается в точность float. Надо было попробовать XGB с MAE, но не хватило времени.

### Очистка данных
1. Как известно, MSE сильно реагирует на выбросы, поэтому необъяснимо неукладывающиеся в общую картину строчки удалялись. Естественно, улучшает cv на почищенных данных. Впрочем, на контрольных тоже.
   * Удалялись в обоих вариантах
2. В выборке f1-f30 дублируются f31-f60. Алгоритму это не вредит (хотя, оптимальные параметры будут разными), но времени работы вредит однозначно.
   * Удалялись
3. Есть 50 строчек с нулевыми признаками. И разными y (первой неделей). Возможно, какое-то обучение на таких данных полезно, но все метрики без этих 50 строк улучшились. 
   * Только для XGB

### Работа с признаками
1. Были попытки сделать что-нибудь с item_id. Например, просто удалить. Бинаризовать разными способами (остаток от деления, двоичная запись, полный перебор). Пытался заменить некоторой статистикой из fXX. Большая часть идей ухудшала результат.
2. Введен признак "количество недель в продаже", равный 30-"число ведущих нулевых признаков". Улучшал качество, но не очень сильно.
   * Только для XGB
3. Признак shift не использовался (ниже)

### Только на неделю вперед
Запросы характеризуются только id товара и неделей. Shift является лишь связью с другими параметрами. В тестовой выборке всего три недели, и для каждой из них приведен запрос со сдвигом 1. Остальные запросы - просто дубли со сдвинутыми признаками.

Обучающая выборка перестраивалась таким образом, чтобы параметры отставали от **y** ровно на одну неделю (calc_date). Лишние строчки удалялись, и алгоритм учился работать только с ближайшей неделей.

Если бы так было сделать нельзя, я бы все равно попробовал обучать три разных алгоритма, т.к. сильно меняется роль параметров, а обучающая выборка -- тройной дубль себя.

Таким образом, особенности тестовой выборки использовались трижды: непосредственно для ответа, для проверки и для упрощения задачи. 